In [ ]:
import tensorflow as tf
import pickle
import os
import numpy as np
import scipy.misc
import cv2
import matplotlib.pyplot as plt
import random

from mpl_toolkits.mplot3d import Axes3D

In [ ]:
import loader
import model
from general import *

In [ ]:
eloader = loader.Data('evaluation')

In [ ]:
imgs, masks, depths, annos = eloader.load_batch(8)

# crop and resize test

In [ ]:
imgs, masks, depths, annos = eloader.load_batch(8)
for idx in range(2):

    plt.imshow(imgs[idx])
    plt.show()

    hands = (np.ones(masks.shape) < masks).astype(int)
    plt.imshow(hands[idx,:,:,0], 'gray')
    plt.show()
    
    bb_list, crsize_list, offset_list = crop_and_resize(imgs, binary_mask)

    oh = tf.cast(tf.stack(offset_list)[:,0], tf.int32)
    ow = tf.cast(tf.stack(offset_list)[:,1], tf.int32)

    th = tf.cast(tf.stack(crsize_list)[:,0], tf.int32)
    tw = tf.cast(tf.stack(crsize_list)[:,1], tf.int32)
    
    cr_imgs = sess.run(tf.image.crop_to_bounding_box(imgs[idx],
                                                 oh[idx],
                                                 ow[idx],
                                                 th[idx],
                                                 tw[idx]))
    plt.imshow(cr_imgs)
    plt.show()

In [ ]:
binary_mask = hands[:,:,:,0]

In [ ]:
imgs.shape

In [ ]:
bb_list, crsize_list, offset_list = crop_and_resize(imgs, binary_mask)

oh = tf.cast(tf.stack(offset_list)[:,0], tf.int32)
ow = tf.cast(tf.stack(offset_list)[:,1], tf.int32)

th = tf.cast(tf.stack(crsize_list)[:,0], tf.int32)
tw = tf.cast(tf.stack(crsize_list)[:,1], tf.int32)

In [ ]:
cr_imgs = sess.run(tf.image.crop_to_bounding_box(imgs[idx],
                                                 oh[idx],
                                                 ow[idx],
                                                 th[idx],
                                                 tw[idx]))
plt.imshow(cr_imgs)
plt.show()

In [ ]:
sess = tf.Session()

In [ ]:
def crop_and_resize(imgs, binary_mask):
    # implement
    imgs_cr = imgs
    crop_offset = tf.ones([8, 2])
    crop_scale = tf.ones([8, 1])

    binary_mask = tf.cast(binary_mask, tf.int32)
    binary_mask = tf.equal(binary_mask, 1)
    s = binary_mask.get_shape().as_list()

    x_range = tf.expand_dims(tf.range(s[1]), 1)
    y_range = tf.expand_dims(tf.range(s[2]), 0)
    X = tf.tile(x_range, [1, s[2]])
    Y = tf.tile(y_range, [s[1], 1])

    # bounding box
    bb_list = []
    crop_size_list = []
    offset_list = []

    for i in range(s[0]):
        X_masked = tf.cast(tf.boolean_mask(X, binary_mask[i, :, :]), tf.float32)
        Y_masked = tf.cast(tf.boolean_mask(Y, binary_mask[i, :, :]), tf.float32)

        x_min = tf.reduce_min(X_masked)
        x_max = tf.reduce_max(X_masked)
        y_min = tf.reduce_min(Y_masked)
        y_max = tf.reduce_max(Y_masked)

        start = tf.stack([x_min, y_min])
        end = tf.stack([x_max, y_max])
        bb = tf.stack([start, end], 1)
        # bb = [x_min, y_min, x_max, y_max]
        
        offset_list.append([x_min, y_min])
        bb_list.append(bb)

        crop_size_x = x_max - x_min
        crop_size_y = y_max - y_min

        crop_size_list.append([crop_size_x, crop_size_y])
        
    return bb_list, crop_size_list, offset_list
    # return imgs_cr, crop_offset, crop_scale

In [ ]:
config_test = {
    'ID' : 'test_model',
    'n_iter' : 20000,
    'n_prt' : 100,
    'input_h' : 320,
    'input_w' : 320,
    'input_ch' : 3,
    'n_output' : 10,
    'n_batch' : 8,
    'n_save' : 1000,
    'n_history' : 50,
    'LR' : 1e-5,
    'random_crop' : False,
    'training' : False,
}

In [ ]:
for key, val in annos.items():
    print(key, val.shape)

In [ ]:
tf.range(320)

In [ ]:
x_range = tf.expand_dims(tf.range(320), 1)
x_range

In [ ]:
X = tf.tile(x_range, [1, 320])
X

In [ ]:
tf.stack?

In [ ]:
a = [[1,2,3],[4,5,6]]
tf.stack(a)

In [ ]:
tf.ones([8,320,320,1])[:,:,:,0]

In [ ]:
tf.tile?ㅁ

In [ ]:
annos['uv'].shape

In [ ]:
annos.keys()

In [ ]:
a = tf.ones([8,320,320,3])
b = a.get_shape()

In [ ]:
_, h, w, _ = b.as_list()

In [ ]:
type(h)

In [ ]:
x = tf.ones([8,32,32,2])
imgs = tf.ones([8,320,320,3])

In [ ]:
_, H, W, _ = imgs.get_shape().as_list()
tf.image.resize_images(x, [H,W])

In [ ]:
config_test = {
    'ID' : 'test_model',
    'n_iter' : 20000,
    'n_prt' : 100,
    'input_h' : 320,
    'input_w' : 320,
    'input_ch' : 3,
    'n_output' : 10,
    'n_batch' : 8,
    'n_save' : 1000,
    'n_history' : 50,
    'LR' : 1e-5,
    'random_crop' : False,
    'training' : False,
}

In [ ]:
SegNet_test = model.Hand3DPoseNet(config=config_test)

# pose 2d?

In [ ]:
sess = tf.Session()

In [ ]:
annos[0]

In [ ]:
evaluation_loader = loader.Data('evaluation')

In [ ]:
imgs, masks, depths, annos = evaluation_loader.load_batch(8)
hand_masks = (np.ones(masks.shape) < masks).astype(int)
hand_masks = hand_masks[:,:,:,2:]

plt.imshow(hand_masks[0,:,:,0], 'gray')
plt.show()

ud = tf.image.flip_up_down(hand_masks)
rl = tf.image.flip_left_right(ud)
flipped = sess.run(rl)

plt.imshow(flipped[0,:,:,0], 'gray')
plt.show()

In [ ]:
th = tf.argmax(hand_masks, axis=1)
th = tf.argmax(th, axis=1)
tw = tf.argmax(hand_masks, axis=2)
tw = tf.argmax(tw, axis=1)

flipped = tf.image.flip_left_right(tf.image.flip_up_down(hand_masks))

bh = tf.argmax(flipped, axis=1)
bh = tf.argmax(bh, axis=1)
bw = tf.argmax(flipped, axis=2)
bw = tf.argmax(bw, axis=1)

th = sess.run(th)[:,0]
tw = sess.run(tw)[:,0]
bh = sess.run(bh)[:,0]
bw = sess.run(bw)[:,0]

idx = 0

print(th[idx], tw[idx], bh[idx], bw[idx])

In [ ]:
plt.imshow(hand_masks[0,th[idx]:,:,0], 'gray')
plt.show()

# calculate center?

In [ ]:
imgs, masks, depths, annos = evaluation_loader.load_batch(8)
hand_masks = (np.ones(masks.shape) < masks).astype(int)
hand_masks = hand_masks[:,:,:,2:]

crop_size = 256
image = tf.constant(hand_masks, tf.float32)
hand_scoremap = tf.constant(hand_masks, tf.float32)

In [ ]:
hand_mask = single_obj_scoremap(hand_scoremap)
center, _, crop_size_best = calc_center_bb(hand_mask)
crop_size_best *= 1.25
scale_crop = tf.minimum(tf.maximum(crop_size / crop_size_best, 0.25), 5.0)

image_crop = crop_image_from_xy(image, center, crop_size, scale=scale_crop)

In [ ]:
t = sess.run(image_crop)

In [ ]:
idx = 4

plt.imshow(imgs[idx])
plt.show()

plt.imshow(hand_masks[idx,:,:,0], 'gray')
plt.show()

plt.imshow(t[idx,:,:,0],'gray')
plt.show()

# with my model?

In [ ]:
config2 = {
    'ID' : 'test_sample',
    'n_iter' : 10000,
    'n_prt' : 100,
    'input_h' : 320,
    'input_w' : 320,
    'input_ch' : 3,
    'n_output' : 10,
    'n_batch' : 8,
    'n_save' : 1000,
    'n_history' : 50,
    'LR' : 1e-6,
    'random_crop' : False
}

In [ ]:
SegNet = model.Hand3DPoseNet(config=config2)
SegNet.load('./test_handseg/checkpoint/test_handseg_20000')

In [ ]:
evaluation_loader = loader.Data('evaluation')

In [ ]:
imgs, masks, depths, annos = evaluation_loader.load_batch(8)
mask_pred = SegNet.sess.run(SegNet.hand_seg_pred, feed_dict = {
    SegNet.imgs : imgs,
    SegNet.masks : masks,
    SegNet.depths : depths
})

mp = np.argmax(mask_pred, axis=3)

for idx in range(8):

    plt.imshow(imgs[idx])
    plt.show()

    hands = (np.ones(masks.shape) < masks).astype(int)
    plt.imshow(hands[idx,:,:,0], 'gray')
    plt.show()

    plt.imshow(mp[idx], 'gray')
    plt.show()

In [ ]:
mask_pred.shape

In [ ]:
sess = tf.Session()

In [ ]:
hand_masks = (np.ones(masks.shape) < masks).astype(int)
hand_masks = hand_masks[:,:,:,2:]

In [ ]:
crop_size = 256
image = tf.constant(imgs, tf.float32)
hand_scoremap = tf.constant(mask_pred, tf.float32)
mp = np.argmax(mask_pred, axis=3)

In [ ]:
hand_mask = single_obj_scoremap(hand_scoremap)
center, _, crop_size_best = calc_center_bb(hand_mask)
crop_size_best *= 1.25
scale_crop = tf.minimum(tf.maximum(crop_size / crop_size_best, 0.25), 5.0)

image_crop = crop_image_from_xy(image, center, crop_size, scale=scale_crop)

t = sess.run(image_crop)

In [ ]:
idx = 4

plt.imshow(imgs[idx])
plt.show()

plt.imshow(mp[idx], 'gray')
plt.show()

plt.imshow(t[idx,:,:,0],'gray')
plt.show()

In [ ]:
a = tf.ones([8,32,32,512])
b = tf.ones([8,32,32,21])

In [ ]:
tf.nn.dropout?

In [ ]:
z1 = tf.zeros([8,1])
o1 = tf.ones([8,1])
tf.concat([z1,o1],axis=1)

In [ ]:
sess.run(tf.concat([z1,o1],axis=1))

In [ ]:
tf.layers.flatten(a)

In [ ]:
tf.concat([tf.layers.flatten(a), tf.concat([z1,o1],axis=1)], axis=1)

In [ ]:
tf.concat([a,b], axis=3)

In [ ]:
tf.concat(tf.concat([a,b], axis=3), tf.constant([1,0], tf.float32))

In [ ]:
annos[0]

# Let's see data

In [ ]:
import pickle
import os
import numpy as np
import scipy.misc
import cv2
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
set = 'evaluation'
f1 = open('./RHD_published_v2/{}/anno_{}.pickle'.format(set,set), 'rb')
anno_all = pickle.load(f1)
f1.close()

In [ ]:
path_to_db = './RHD_published_v2/'
set = 'evaluation'

In [ ]:
sample_id = 4

image = cv2.imread(os.path.join(path_to_db, set , 'color', '%.5d.png' % sample_id))
mask = cv2.imread(os.path.join(path_to_db, set , 'mask', '%.5d.png' % sample_id))
depth = cv2.imread(os.path.join(path_to_db, set , 'depth', '%.5d.png' % sample_id))

image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
mask = cv2.cvtColor(mask, cv2.COLOR_BGR2RGB)
depth = cv2.cvtColor(depth, cv2.COLOR_BGR2RGB)

hand = (np.ones(mask.shape)*1 < mask).astype(int)
hand *= 240
plt.imshow(hand)
plt.show()
anno = anno_all[sample_id]

In [ ]:
anno['uv_vis']

In [ ]:
anno = anno_all[sample_id]

# get info from annotation dictionary
kp_coord_uv = anno['uv_vis'][:, :2]  # u, v coordinates of 42 hand keypoints, pixel
kp_visible = (anno['uv_vis'][:, 2] == 1)  # visibility of the keypoints, boolean
kp_coord_xyz = anno['xyz']  # x, y, z coordinates of the keypoints, in meters
camera_intrinsic_matrix = anno['K']  # matrix containing intrinsic parameters

# Project world coordinates into the camera frame
kp_coord_uv_proj = np.matmul(kp_coord_xyz, np.transpose(camera_intrinsic_matrix))
kp_coord_uv_proj = kp_coord_uv_proj[:, :2] / kp_coord_uv_proj[:, 2:]

In [ ]:
kp_coord_uv.shape

In [ ]:
kp_coord_uv[kp_visible].shape

In [ ]:
# Visualize data
fig = plt.figure(1, figsize=(15,15))
ax1 = fig.add_subplot('221')
ax2 = fig.add_subplot('222')
ax3 = fig.add_subplot('223')
ax4 = fig.add_subplot('224', projection='3d')

ax1.imshow(image)
ax1.plot(kp_coord_uv[kp_visible, 0], kp_coord_uv[kp_visible, 1], 'ro')
ax1.plot(kp_coord_uv_proj[kp_visible, 0], kp_coord_uv_proj[kp_visible, 1], 'gx')
ax2.imshow(depth)
ax3.imshow(mask)
ax4.scatter(kp_coord_xyz[kp_visible, 0], kp_coord_xyz[kp_visible, 1], kp_coord_xyz[kp_visible, 2])
# ax4.view_init(azim=-90.0, elev=-90.0)  # aligns the 3d coord with the camera view
# plot_hand_3d(kp_coord_xyz, ax4, color_fixed=None, linewidth='1')
# plot_hand(kp_coord_xyz, ax4, color_fixed=None, linewidth='1')
ax4.set_xlabel('x')
ax4.set_ylabel('y')
ax4.set_zlabel('z')

plt.show()


In [ ]:
len(anno_all)

In [ ]:
depth

In [ ]:
plt.imshow(mask*7)
plt.show()

In [ ]:
print(image.shape)
print(mask.shape)
print(depth.shape)

In [ ]:
plt.imshow(image)
plt.show()
plt.imshow(mask)
plt.show()
plt.imshow(depth)
plt.show()

In [ ]:
plt.imshow(image)
plt.show()
plt.imshow(mask)
plt.show()
plt.imshow(depth)
plt.show()

In [ ]:
def depth_two_uint8_to_float(top_bits, bottom_bits):
    """ Converts a RGB-coded depth into float valued depth. """
    depth_map = (top_bits * 2**8 + bottom_bits).astype('float32')
    depth_map /= float(2**16 - 1)
    depth_map *= 5.0
    return depth_map

In [ ]:
depth = depth_two_uint8_to_float(depth[:, :, 0], depth[:, :, 1])

In [ ]:
depth

In [ ]:
kp_coord_xyz

In [ ]:
np.max(mask)

In [ ]:
anno.keys()

In [ ]:
anno = anno_all[sample_id]

# get info from annotation dictionary
kp_coord_uv = anno['uv_vis'][:, :2]  # u, v coordinates of 42 hand keypoints, pixel
kp_visible = (anno['uv_vis'][:, 2] == 1)  # visibility of the keypoints, boolean
kp_coord_xyz = anno['xyz']  # x, y, z coordinates of the keypoints, in meters
camera_intrinsic_matrix = anno['K']  # matrix containing intrinsic parameters

# Project world coordinates into the camera frame
kp_coord_uv_proj = np.matmul(kp_coord_xyz, np.transpose(camera_intrinsic_matrix))
kp_coord_uv_proj = kp_coord_uv_proj[:, :2] / kp_coord_uv_proj[:, 2:]

# Visualize data
fig = plt.figure(1, figsize=(15,15))
ax1 = fig.add_subplot('221')
ax2 = fig.add_subplot('222')
ax3 = fig.add_subplot('223')
ax4 = fig.add_subplot('224', projection='3d')

ax1.imshow(image)
ax1.plot(kp_coord_uv[kp_visible, 0], kp_coord_uv[kp_visible, 1], 'ro')
ax1.plot(kp_coord_uv_proj[kp_visible, 0], kp_coord_uv_proj[kp_visible, 1], 'gx')
ax2.imshow(depth)
ax3.imshow(mask)
ax4.scatter(kp_coord_xyz[kp_visible, 0], kp_coord_xyz[kp_visible, 1], kp_coord_xyz[kp_visible, 2])
# ax4.view_init(azim=-90.0, elev=-90.0)  # aligns the 3d coord with the camera view
# plot_hand_3d(kp_coord_xyz, ax4, color_fixed=None, linewidth='1')
# plot_hand(kp_coord_xyz, ax4, color_fixed=None, linewidth='1')
ax4.set_xlabel('x')
ax4.set_ylabel('y')
ax4.set_zlabel('z')

plt.show()


In [ ]:
coord_hw = kp_coord_xyz
fig = plt.figure(1, figsize=(10,10))
ax1 = fig.add_subplot(221)
ax2 = fig.add_subplot(222)
ax3 = fig.add_subplot(223)
ax4 = fig.add_subplot(224, projection='3d')
ax1.imshow(image)
plot_hand(coord_hw, ax1)
ax2.imshow(image_crop_v)
plot_hand(coord_hw_crop, ax2)
ax3.imshow(np.argmax(hand_scoremap_v, 2))
plot_hand_3d(keypoint_coord3d_v, ax4)
ax4.view_init(azim=-90.0, elev=-90.0)  # aligns the 3d coord with the camera view
ax4.set_xlim([-3, 3])
ax4.set_ylim([-3, 1])
ax4.set_zlim([-3, 3])
plt.show()

In [ ]:
def plot_hand(coords_hw, axis, color_fixed=None, linewidth='1'):
    """ Plots a hand stick figure into a matplotlib figure. """
    colors = np.array([[0., 0., 0.5],
                       [0., 0., 0.73172906],
                       [0., 0., 0.96345811],
                       [0., 0.12745098, 1.],
                       [0., 0.33137255, 1.],
                       [0., 0.55098039, 1.],
                       [0., 0.75490196, 1.],
                       [0.06008855, 0.9745098, 0.90765338],
                       [0.22454143, 1., 0.74320051],
                       [0.40164453, 1., 0.56609741],
                       [0.56609741, 1., 0.40164453],
                       [0.74320051, 1., 0.22454143],
                       [0.90765338, 1., 0.06008855],
                       [1., 0.82861293, 0.],
                       [1., 0.63979666, 0.],
                       [1., 0.43645606, 0.],
                       [1., 0.2476398, 0.],
                       [0.96345811, 0.0442992, 0.],
                       [0.73172906, 0., 0.],
                       [0.5, 0., 0.]])

    # define connections and colors of the bones
    bones = [((0, 4), colors[0, :]),
             ((4, 3), colors[1, :]),
             ((3, 2), colors[2, :]),
             ((2, 1), colors[3, :]),

             ((0, 8), colors[4, :]),
             ((8, 7), colors[5, :]),
             ((7, 6), colors[6, :]),
             ((6, 5), colors[7, :]),

             ((0, 12), colors[8, :]),
             ((12, 11), colors[9, :]),
             ((11, 10), colors[10, :]),
             ((10, 9), colors[11, :]),

             ((0, 16), colors[12, :]),
             ((16, 15), colors[13, :]),
             ((15, 14), colors[14, :]),
             ((14, 13), colors[15, :]),

             ((0, 20), colors[16, :]),
             ((20, 19), colors[17, :]),
             ((19, 18), colors[18, :]),
             ((18, 17), colors[19, :])]

    for connection, color in bones:
        coord1 = coords_hw[connection[0], :]
        coord2 = coords_hw[connection[1], :]
        coords = np.stack([coord1, coord2])
        if color_fixed is None:
            axis.plot(coords[:, 1], coords[:, 0], color=color, linewidth=linewidth)
        else:
            axis.plot(coords[:, 1], coords[:, 0], color_fixed, linewidth=linewidth)

In [ ]:
def plot_hand_3d(coords_xyz, axis, color_fixed=None, linewidth='1'):
    """ Plots a hand stick figure into a matplotlib figure. """
    colors = np.array([[0., 0., 0.5],
                       [0., 0., 0.73172906],
                       [0., 0., 0.96345811],
                       [0., 0.12745098, 1.],
                       [0., 0.33137255, 1.],
                       [0., 0.55098039, 1.],
                       [0., 0.75490196, 1.],
                       [0.06008855, 0.9745098, 0.90765338],
                       [0.22454143, 1., 0.74320051],
                       [0.40164453, 1., 0.56609741],
                       [0.56609741, 1., 0.40164453],
                       [0.74320051, 1., 0.22454143],
                       [0.90765338, 1., 0.06008855],
                       [1., 0.82861293, 0.],
                       [1., 0.63979666, 0.],
                       [1., 0.43645606, 0.],
                       [1., 0.2476398, 0.],
                       [0.96345811, 0.0442992, 0.],
                       [0.73172906, 0., 0.],
                       [0.5, 0., 0.]])

    # define connections and colors of the bones
    bones = [((0, 4), colors[0, :]),
             ((4, 3), colors[1, :]),
             ((3, 2), colors[2, :]),
             ((2, 1), colors[3, :]),

             ((0, 8), colors[4, :]),
             ((8, 7), colors[5, :]),
             ((7, 6), colors[6, :]),
             ((6, 5), colors[7, :]),

             ((0, 12), colors[8, :]),
             ((12, 11), colors[9, :]),
             ((11, 10), colors[10, :]),
             ((10, 9), colors[11, :]),

             ((0, 16), colors[12, :]),
             ((16, 15), colors[13, :]),
             ((15, 14), colors[14, :]),
             ((14, 13), colors[15, :]),

             ((0, 20), colors[16, :]),
             ((20, 19), colors[17, :]),
             ((19, 18), colors[18, :]),
             ((18, 17), colors[19, :])]

    for connection, color in bones:
        coord1 = coords_xyz[connection[0], :]
        coord2 = coords_xyz[connection[1], :]
        coords = np.stack([coord1, coord2])
        if color_fixed is None:
            axis.plot(coords[:, 0], coords[:, 1], coords[:, 2], color=color, linewidth=linewidth)
        else:
            axis.plot(coords[:, 0], coords[:, 1], coords[:, 2], color_fixed, linewidth=linewidth)

    axis.view_init(azim=-90., elev=90.)

In [ ]:
fig = plt.figure(1, figsize=(20,20))
ax1 = fig.add_subplot('221')
ax2 = fig.add_subplot('222')
ax3 = fig.add_subplot('223')
ax4 = fig.add_subplot('224', projection='3d')
plot_hand_3d(kp_coord_xyz, ax4, color_fixed=None, linewidth='1')
plot_hand(kp_coord_xyz, ax3, color_fixed=None, linewidth='1')

In [ ]:
plot_hand_3d(kp_coord_xyz, ax4, color_fixed=None, linewidth='1')

In [ ]:
plt.show()

# general 실험

In [ ]:
ops = NetworkOps

In [ ]:
def inference_pose2d(image_crop, train=False):
    """ PoseNet: Given an image it detects the 2D hand keypoints.
        The image should already contain a rather tightly cropped hand.
        Inputs:
            image: [B, H, W, 3] tf.float32 tensor, Image with mean subtracted
            train: bool, True in case weights should be trainable
        Outputs:
            scoremap_list_large: list of [B, 256, 256, 21] tf.float32 tensor, Scores for the hand keypoints
    """
    with tf.variable_scope('PoseNet2D'):
        scoremap_list = list()
        layers_per_block = [2, 2, 4, 2]
        out_chan_list = [64, 128, 256, 512]
        pool_list = [True, True, True, False]

        # learn some feature representation, that describes the image content well
        x = image_crop
        for block_id, (layer_num, chan_num, pool) in enumerate(zip(layers_per_block, out_chan_list, pool_list), 1):
            for layer_id in range(layer_num):
                x = ops.conv_relu(x, 'conv%d_%d' % (block_id, layer_id+1), kernel_size=3, stride=1, out_chan=chan_num, trainable=train)
            if pool:
                x = ops.max_pool(x, 'pool%d' % block_id)

        x = ops.conv_relu(x, 'conv4_3', kernel_size=3, stride=1, out_chan=256, trainable=train)
        x = ops.conv_relu(x, 'conv4_4', kernel_size=3, stride=1, out_chan=256, trainable=train)
        x = ops.conv_relu(x, 'conv4_5', kernel_size=3, stride=1, out_chan=256, trainable=train)
        x = ops.conv_relu(x, 'conv4_6', kernel_size=3, stride=1, out_chan=256, trainable=train)
        encoding = ops.conv_relu(x, 'conv4_7', kernel_size=3, stride=1, out_chan=128, trainable=train)

        # use encoding to detect initial scoremap
        x = ops.conv_relu(encoding, 'conv5_1', kernel_size=1, stride=1, out_chan=512, trainable=train)
        scoremap = ops.conv(x, 'conv5_2', kernel_size=1, stride=1, out_chan=21, trainable=train)
        scoremap_list.append(scoremap)

        # iterate recurrent part a couple of times
        layers_per_recurrent_unit = 5
        num_recurrent_units = 2
        for pass_id in range(num_recurrent_units):
            x = tf.concat([scoremap_list[-1], encoding], 3)
            for rec_id in range(layers_per_recurrent_unit):
                x = ops.conv_relu(x, 'conv%d_%d' % (pass_id+6, rec_id+1), kernel_size=7, stride=1, out_chan=128, trainable=train)
            x = ops.conv_relu(x, 'conv%d_6' % (pass_id+6), kernel_size=1, stride=1, out_chan=128, trainable=train)
            scoremap = ops.conv(x, 'conv%d_7' % (pass_id+6), kernel_size=1, stride=1, out_chan=21, trainable=train)
            scoremap_list.append(scoremap)

        scoremap_list_large = scoremap_list

    return scoremap_list_large

In [ ]:
image_crop = tf.ones([8,320,320,3])
k = inference_pose2d(image_crop)

In [ ]:
sess = tf.Session()

In [ ]:
k[-1]

In [ ]:
tf.image.resize_images(k[-1], [256,256])

In [ ]:
def _gaussian_kernel(kernel_size, sigma, n_channels, dtype):
    x = tf.range(-kernel_size // 2 + 1, kernel_size // 2 + 1, dtype=dtype)
    g = tf.math.exp(-(tf.pow(x, 2) / (2 * tf.pow(tf.cast(sigma, dtype), 2))))
    g_norm2d = tf.pow(tf.reduce_sum(g), 2)
    g_kernel = tf.tensordot(g, g, axes=0) / g_norm2d
    g_kernel = tf.expand_dims(g_kernel, axis=-1)
    return tf.expand_dims(tf.tile(g_kernel, (1, 1, n_channels)), axis=-1)

In [ ]:
_gaussian_kernel(5, 2, 3, tf.float32)

In [ ]:
gk = sess.run(_gaussian_kernel(5, 2, 3, tf.float32)

In [ ]:
gk[:,:,0,0]

In [ ]:
def create_multiple_gaussian_map(coords_uv, output_size, sigma, valid_vec=None):
    """ Creates a map of size (output_shape[0], output_shape[1]) at (center[0], center[1])
        with variance sigma for multiple coordinates."""
    with tf.name_scope('create_multiple_gaussian_map'):
        sigma = tf.cast(sigma, tf.float32)
        assert len(output_size) == 2
        s = coords_uv.get_shape().as_list()
        coords_uv = tf.cast(coords_uv, tf.int32)
        if valid_vec is not None:
            valid_vec = tf.cast(valid_vec, tf.float32)
            valid_vec = tf.squeeze(valid_vec)
            cond_val = tf.greater(valid_vec, 0.5)
        else:
            cond_val = tf.ones_like(coords_uv[:, 0], dtype=tf.float32)
            cond_val = tf.greater(cond_val, 0.5)

        cond_1_in = tf.logical_and(tf.less(coords_uv[:, 0], output_size[0]-1), tf.greater(coords_uv[:, 0], 0))
        cond_2_in = tf.logical_and(tf.less(coords_uv[:, 1], output_size[1]-1), tf.greater(coords_uv[:, 1], 0))
        cond_in = tf.logical_and(cond_1_in, cond_2_in)
        cond = tf.logical_and(cond_val, cond_in)

        coords_uv = tf.cast(coords_uv, tf.float32)

        # create meshgrid
        x_range = tf.expand_dims(tf.range(output_size[0]), 1)
        y_range = tf.expand_dims(tf.range(output_size[1]), 0)

        X = tf.cast(tf.tile(x_range, [1, output_size[1]]), tf.float32)
        Y = tf.cast(tf.tile(y_range, [output_size[0], 1]), tf.float32)

        X.set_shape((output_size[0], output_size[1]))
        Y.set_shape((output_size[0], output_size[1]))

        X = tf.expand_dims(X, -1)
        Y = tf.expand_dims(Y, -1)

        X_b = tf.tile(X, [1, 1, s[0]])
        Y_b = tf.tile(Y, [1, 1, s[0]])

        X_b -= coords_uv[:, 0]
        Y_b -= coords_uv[:, 1]

        dist = tf.square(X_b) + tf.square(Y_b)

        scoremap = tf.exp(-dist / tf.square(sigma)) * tf.cast(cond, tf.float32)

        return scoremap


In [ ]:
coords_uv = [[0,0],[10,10],[15,15],[20,20]]
coords_uv = tf.constant(coords_uv, tf.float32)
coords_uv

In [ ]:
coords_uv = tf.stack([anno['uv_vis'][:,0], anno['uv_vis'][:,1]], -1)
coords_uv = tf.cast(coords_uv, tf.int32)
coords_uv

In [ ]:
valid_vec

In [ ]:
valid_vec = anno['uv_vis'][:,2]
valid_vec.shape

In [ ]:
create_multiple_gaussian_map(coords_uv, [256,256], 25, [1,1,1,1])

In [ ]:
a = sess.run(create_multiple_gaussian_map(coords_uv, [256,256], 25, valid_vec))

In [ ]:
valid_vec

In [ ]:
a.shape

In [ ]:
np.sum(a[:,:,4])

In [ ]:
tf.gather?

In [ ]:
anno['uv_vis'][:,0]

In [ ]:
tf.cast(kp_visible, tf.int32)

In [ ]:
u = np.array(kp_coord_uv[kp_visible, 0]).astype(int)
v = np.array(kp_coord_uv[kp_visible, 1]).astype(int)

tfkp = tf.stack([u,v],-1)
tfkp

In [ ]:
sess.run(tfkp)

In [ ]:
kp_coord_uv

In [ ]:
tfkp[0:1]

In [ ]:
a = tf.sparse_to_dense(tfkp[2:3], [256,256], sparse_values=1.0)
b = sess.run(a)

In [ ]:
tf.expand_dims(a,2)

In [ ]:
tf.nn.depthwise_conv2d(tf.expand_dims(a,2), _gaussian_kernel, [1,1,1,1], 'SAME')

In [ ]:
np.where(b==1)

In [ ]:
# get info from annotation dictionary
kp_coord_uv = anno['uv_vis'][:, :2]  # u, v coordinates of 42 hand keypoints, pixel
kp_visible = (anno['uv_vis'][:, 2] == 1)  # visibility of the keypoints, boolean
kp_coord_xyz = anno['xyz']  # x, y, z coordinates of the keypoints, in meters
camera_intrinsic_matrix = anno['K']  # matrix containing intrinsic parameters

# Project world coordinates into the camera frame
kp_coord_uv_proj = np.matmul(kp_coord_xyz, np.transpose(camera_intrinsic_matrix))
kp_coord_uv_proj = kp_coord_uv_proj[:, :2] / kp_coord_uv_proj[:, 2:]

# Visualize data
fig = plt.figure(1, figsize=(15,15))
ax1 = fig.add_subplot('221')
ax2 = fig.add_subplot('222')
ax3 = fig.add_subplot('223')
ax4 = fig.add_subplot('224', projection='3d')

ax1.imshow(image)
ax1.plot(kp_coord_uv[kp_visible, 0], kp_coord_uv[kp_visible, 1], 'ro')
ax1.plot(kp_coord_uv_proj[kp_visible, 0], kp_coord_uv_proj[kp_visible, 1], 'gx')
ax2.imshow(depth)
ax3.imshow(mask)
ax4.scatter(kp_coord_xyz[kp_visible, 0], kp_coord_xyz[kp_visible, 1], kp_coord_xyz[kp_visible, 2])
# ax4.view_init(azim=-90.0, elev=-90.0)  # aligns the 3d coord with the camera view
# plot_hand_3d(kp_coord_xyz, ax4, color_fixed=None, linewidth='1')
# plot_hand(kp_coord_xyz, ax4, color_fixed=None, linewidth='1')
ax4.set_xlabel('x')
ax4.set_ylabel('y')
ax4.set_zlabel('z')

plt.show()
